# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv = "../WeatherPy/clean_city_data.csv"
clean_city_data_df = pd.read_csv(csv)
clean_city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,the valley,18.22,-63.06,87.80,58,20,17.22,AI,1595444535
1,coihaique,-45.58,-72.07,33.80,74,75,4.70,CL,1595445069
2,cape town,-33.93,18.42,57.00,93,0,8.05,ZA,1595444767
3,hermanus,-34.42,19.23,54.00,94,0,5.37,ZA,1595445072
4,thompson,55.74,-97.86,78.80,47,75,9.17,CA,1595445066
...,...,...,...,...,...,...,...,...,...
533,ushtobe,45.25,77.98,72.50,45,0,8.72,KZ,1595445388
534,san giovanni rotondo,41.70,15.72,84.20,45,5,10.29,IT,1595445388
535,shirokiy,49.76,129.52,61.34,86,100,4.38,RU,1595445388
536,leirvik,59.78,5.50,60.01,69,25,1.99,NO,1595445388


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
hum = clean_city_data_df['Humidity'].astype(float)
locations = clean_city_data_df[['Lat', 'Lng']]

fig = gmaps.figure(center=(0,0), zoom_level=2)
heatmap = gmaps.heatmap_layer(locations, weights=hum, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Max temp between 50 and 75 degrees and humidity lower than 50%

my_destinations = clean_city_data_df.loc[(clean_city_data_df['Max Temp'] <= 75) & (clean_city_data_df['Max Temp'] >= 50) & (clean_city_data_df['Humidity'] <= 50)]
my_destinations = my_destinations.dropna()
my_destinations

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,creel,27.75,-107.63,70.97,47,100,5.28,MX,1595445308
69,narsaq,60.92,-46.05,68.00,24,40,29.97,GL,1595445060
120,aksu,41.12,80.26,73.78,42,98,8.21,CN,1595445323
151,krasnoarmeysk,51.02,45.70,72.00,47,28,1.01,RU,1595445328
155,zabol,32.25,67.25,72.30,20,0,11.68,AF,1595445329
172,ilulissat,69.22,-51.10,55.40,50,40,5.82,GL,1595445058
199,mahajanga,-15.72,46.32,73.40,49,20,11.41,MG,1595445335
208,shakawe,-18.37,21.85,59.02,34,0,4.92,BW,1595445337
237,ust-nera,64.57,143.20,67.10,49,96,3.04,RU,1595445169
240,yumen,40.28,97.20,71.74,44,100,7.38,CN,1595445342


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
my_destinations['Hotel Name'] = ""

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "key": g_key,
    "radius": 5000,
    "types": "logding",
    "keyword": "hotel"
}

for index, row in my_destinations.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"

    response = requests.get(url, params=params)
    hotels = response.json()
    
    try:
        my_destinations.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except:
        pass

hotel_df = my_destinations.loc[my_destinations['Hotel Name'] != '']
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,creel,27.75,-107.63,70.97,47,100,5.28,MX,1595445308,The Lodge at Creel Hotel & SPA
69,narsaq,60.92,-46.05,68.00,24,40,29.97,GL,1595445060,Igdlo Guesthouse
120,aksu,41.12,80.26,73.78,42,98,8.21,CN,1595445323,Pudong Holiday Hotel
151,krasnoarmeysk,51.02,45.70,72.00,47,28,1.01,RU,1595445328,"Gostinichno-Razvlekatel'nyy Kompleks ""Banya"""
172,ilulissat,69.22,-51.10,55.40,50,40,5.82,GL,1595445058,Hotel Arctic
199,mahajanga,-15.72,46.32,73.40,49,20,11.41,MG,1595445335,Lodge des Terres Blanches
208,shakawe,-18.37,21.85,59.02,34,0,4.92,BW,1595445337,Athoma Guest House
237,ust-nera,64.57,143.20,67.10,49,96,3.04,RU,1595445169,Gostevoy Dom
255,chapais,49.78,-74.85,73.40,43,20,6.93,CA,1595445119,Hôtel Opémiska
371,konstantinovsk,47.58,41.09,71.69,50,0,8.25,RU,1595445363,Петровская Застава


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)
hum = hotel_df['Humidity'].astype(float)

fig = gmaps.figure(center=(0,0), zoom_level=1)
heatmap = gmaps.heatmap_layer(locations, weights=hum, dissipating=False, max_intensity=100, point_radius=1)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heatmap)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))